### Data-cleaning setup

We begin the cleaning workflow by importing **pandas**, the core library we’ll use for loading, inspecting, and transforming the accident dataset.

### Import required library

We load **pandas**, the main Python toolkit for data wrangling and analysis, and give it the conventional alias `pd` for shorter code.

In [ ]:
import pandas as pd

### Load the aviation accident dataset

We start by importing the dataset, which includes all recorded aviation accidents from 1962 to 2023.
The file isn’t encoded in UTF-8, so trying to load it that way causes an error.
Using `encoding="latin1"` instead solves the problem and lets us read in the full dataset.
We then use `df.head()` to take a quick look at the first few rows and get a sense of the data.


In [ ]:
df = pd.read_csv("AviationData.csv", encoding = "latin1") # utf-8 throws an error, so switched to latin1 encoding
df.head() # initial data understanding

### Check dataset dimensions and structure

Before cleaning, it helps to know how large the dataset is and what it contains.
The code below:

i. Prints the **number of rows** and **columns** so we see the overall size.
ii. Calls `df.info()` to show basic data types and missing-value counts.
iii. Lists all column names, giving us a clear inventory of the fields we’ll be working with.


In [ ]:
# Now we need to understand how big the data we're working with is
tuple_shape = df.shape

print(f"Rows: {tuple_shape[0]}")
print(f"Columns: {tuple_shape[1]}")
df.info()
print(df.columns.to_list())

### Convert date columns to true datetime objects

Turning the text-based date columns into pandas `datetime` format lets us later sort, filter, or group by year and month.
Using `errors="coerce"` quietly sets any bad or empty strings to `NaT` (pandas’ “not-a-time” value), so the loop finishes without crashing even if some dates are missing.


In [ ]:
# we convert dates to datetime format (not sure if needed, just for practise)
date_cols = ["Event.Date", "Publication.Date"]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors="coerce")

### Convert key count columns to numeric

These five columns hold numbers (engine count and injury totals) but were read in as text.
`pd.to_numeric(..., errors="coerce")` changes them to proper numeric types and turns any bad entries into `NaN`, which we can handle later.
A quick `dtypes` printout confirms the conversion worked for both the numeric and date columns.


In [ ]:
# Next, convert numeric columns to Numeric values
numeric_cols = ["Number.of.Engines", "Total.Fatal.Injuries", "Total.Serious.Injuries", "Total.Minor.Injuries", "Total.Uninjured"]

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors ="coerce")

# Make sure our conversion worked (it did)
print(df[numeric_cols].dtypes)
print(df[date_cols].dtypes)

### Remove columns that aren’t useful for our risk analysis

The fields listed in `irrelevant_columns` are mostly IDs, location codes, or carrier details that won’t help us judge aircraft safety.
We drop them to keep the dataset focused and easier to work with, then call `df_simple.info()` to confirm the new, slimmer structure.


In [ ]:
# Next, we need to drop colums that won't help our analysis
irrelevant_columns = ["Event.Id", "Accident.Number", "Latitude", "Longitude", "Airport.Code", "Airport.Name", "Registration.Number", "Schedule", "Location", "Air.carrier", "Report.Status", "Country"]
df_simple = df.drop(columns=irrelevant_columns)
df_simple.info()

### Fill missing numbers with each column’s median value

To avoid losing rows that have a few missing injury or engine counts, we fill those gaps with the median of each column.
The median is a safe choice because it isn’t thrown off by extreme accident records.
After filling, we call `df_simple.info()` again to confirm there are no remaining nulls in the numeric columns.

In [ ]:
# We're going to fill the NaN in the numerical columns by entering the median
for i in numeric_cols:
    df_simple[i] = df_simple[i].fillna(df_simple[i].median())

df_simple.info()

### Fill small gaps in key categorical columns

Some rows still have a few missing labels (NaN) in columns like flight purpose, aircraft damage, and weather.
Rather than lose those rows, we replace each missing entry with the **most common value (mode)** for that column.
This keeps the categories consistent and avoids introducing new labels.

In [ ]:
# we'll also fill categorical data with few values missing with the most frequent category
categorical_data = ["Purpose.of.flight", "Aircraft.damage", "Injury.Severity", "Engine.Type", "Weather.Condition", "Amateur.Built"]

for i in categorical_data:
    most_common = df_simple[col].mode(dropna=True)[0] # to get the mode
    df_simple[i].fillna(most_common, inplace=True)

df_simple.info()

### Inspect the raw “Make” values for inconsistencies

Before cleaning, we list the 20 most frequent manufacturer names. This quick peek shows issues like mixed casing, extra spaces, and punctuation that we’ll fix next.

In [ ]:
# There's a naming inconsistency in the names of makes. We have to resolve that
df_simple["Make"].dropna().value_counts().head(20)

### Standardize “Make” names and preview common models

We clean the **Make** column by capitalizing, trimming spaces, and stripping punctuation so each manufacturer is counted once. After that, we check the 20 most common **Model** names to see what aircraft appear most often.

In [ ]:
# Combined all names by making case uniform and removing whitespace 
df_simple["Make"] = df_simple["Make"].str.capitalize().str.strip().str.replace(r"[^\w\s]", "", regex=True)
df_simple["Model"].dropna().value_counts().head(20)

### Keep only rows with both Make and Model

Rows missing either field can’t be tied to a specific aircraft, so we drop them to ensure every record has a valid manufacturer and model for analysis.


In [ ]:
# lastly, we drop every row that doesn't have the make or model
df_simple = df_simple.dropna(subset=["Make", "Model"])
df_simple.info()

We now have a clean dataset!